In [6]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(stringr)
library(data.table)


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [67]:
file_list<-list.files(path = "01B_MAST/", pattern = NULL, all.files = FALSE,
           full.names = FALSE, recursive = FALSE,
           ignore.case = FALSE, include.dirs = FALSE, no.. = FALSE)
print(length(file_list))

[1] 11708


In [68]:
split_list <- strsplit(file_list, "_")
df <- as.data.frame(do.call(rbind, split_list), stringsAsFactors = FALSE)

In [69]:
table(df$V2,df$V1)

    
     BR1001 BR1002 BR1003 BR1004 BR1005 BR1006 BR1007 BR1008 BR1009 BR1010
  Y1     71     67     68     68     66     70     69     68     69     65
  Y2      0     69     67     66     69     68     69     64     68     68
    
     BR1011 BR1012 BR1013 BR1014 BR1015 BR1016 BR1017 BR1018 BR1019 BR1021
  Y1     67     69     68     69     68     65     68     63     69     68
  Y2     65     66     68     69      0     63     69     62     70     68
    
     BR1023 BR1024 BR1025 BR1026 BR1028 BR1030 BR1031 BR1032 BR1033 BR1037
  Y1     66     68     68     67     66     68     66     67     66     68
  Y2     65     63     68     66     67     69     67     62      0     69
    
     BR1040 BR1041 BR1042 BR1043 BR1044 BR1045 BR1047 BR1048 BR1049 BR1050
  Y1     66     68     68     66     66     70     68     70     69     68
  Y2     67     67      0     68      0      0     70     67     65     68
    
     BR1051 BR1052 BR1054 BR1056 BR1057 BR1058 BR1059 BR2002 BR2003 BR2005


In [70]:
table(df$V3,df$V2)

                                          
                                           Y1 Y2
  Activated memory B cell.csv              61 69
  Adaptive NK cell.csv                     92 84
  ASDC.csv                                 81 71
  BaEoMaP cell.csv                         21 16
  C1Q+ CD16 monocyte.csv                   92 84
  CD14+ cDC2.csv                           92 84
  CD27- effector B cell.csv                92 84
  CD27+ effector B cell.csv                92 84
  CD4 MAIT.csv                             85 77
  CD56bright NK cell.csv                   92 84
  CD8 MAIT.csv                             92 84
  CD8aa.csv                                83 73
  CD95 memory B cell.csv                   92 84
  cDC1.csv                                 92 81
  CLP cell.csv                             58 54
  CM CD4 T cell.csv                        92 84
  CM CD8 T cell.csv                        92 84
  CMP cell.csv                             89 82
  Core CD14 monocyte.csv  

# MAST

In [79]:
DEG_FILES<-list.files(path = "01B_MAST/", pattern = ".csv", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE, no.. = FALSE)

In [88]:
de_list<-mclapply(DEG_FILES,function(i){
DT<-read.csv(i,check.names=FALSE)
DT$celltype<- str_extract(i, "(?<=_)[^_]+(?=\\.csv)")
DT$timepoint<- str_match(i, "^[^_]*_[^_]*_([^_]*)")[,2]
 DT$Donor<-   str_extract(i, "(?<=//)[^_]*")
DT<-DT[2:15]
result <- merge(data.table(DT)[contrast=='VisitFlu_Year_1_Day_7' & component=='H',.(primerid, `Pr(>Chisq)`)], # p-values
                     data.table(DT)[contrast=='VisitFlu_Year_1_Day_7' & component=='logFC', .(primerid,coef,Donor,celltype,timepoint,cell_counts_D0,cell_counts_D7)],
                     by='primerid')
result[,coef:=result[,coef]/log(2)]
result[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')]    
},mc.cores=60)

In [90]:
df<-do.call(rbind,de_list)

In [92]:
write.csv(df,'MAST_res.csv')

In [93]:
df1<-read.csv('MAST_res.csv')

In [ ]:
uploadFiles(
  list('/home/jupyter/BRI_Analysis/scRNA/Analysis-Longitudinal_Flu/MAST_res.csv'),
  studySpaceId = '64097865-486d-43b3-8f94-74994e0a72e0',
  title = 'Myeloidcells object uploads--20231107', fileTypes = list('h5#derived'),
  inputFileIds = list('fec489f9-9a74-4635-aa91-d2bf09d1faec')
)